# What's my Home Worth?

Many real estate agents follow a notion of comps (or comparables) when pricing a home. Problem here is that this is often subject to an individual realtors' biases and often only looks at 2-3 properties in the area as a comparison.

What's my home worth is a machine learning model deployed on the Acumos store for pricing your home based on fundamental property characteristics.

More specifically, we survey the intrinsic properties of a home, such as number of bathrooms, number of bedrooms, square footage, and location, as sampled from the Redfin website (www.redfin.com), and allow users to price their home using a large dataset of collected properties.

By no means is this an exhaustive model, but should give users as idea of what their home (or someone else's home) is at least worth in the current market.




In [9]:
import os

pw = os.environ['ACUMOS_PASSWORD']
user = os.environ['ACUMOS_USERNAME']

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.session import AcumosSession

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy.linalg.interpolative as sli
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd
import numpy.linalg as LA
import math
# import xgboost as xgb
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from scipy import stats
# from skimage import color

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.grid_search import GridSearchCV

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [23]:
# Redfin Model (using recently sold to predict house valuations).

# def edit_nan(x, suffix):
#     if x == 'nan':
#         return x + suffix
#     return x
# def mask(df, f):
#     return df[f(df)]

class RedfinModel:
    URL_COL = 'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)'
    DROP_COLS = [URL_COL, 'NEXT OPEN HOUSE START TIME', 'FAVORITE', 'INTERESTED', 'ZIP', 'SALE TYPE',
                 'NEXT OPEN HOUSE END TIME', 'STATUS', 'SOLD DATE', 'MLS#',
                 'SOURCE', 'ADDRESS', 'LATITUDE','LONGITUDE'] # 'LISTING ID'
    ENCODED_COLS = ['CITY', 'STATE', 'PROPERTY TYPE', 'LOCATION']
    
    
    def __init__(self):
        # self.train = self.process_train_data(self.data)
        self.X = None
        self.y = None
        self.df = None
        return
    
    def encode_onehot(self, df, cols):
        """
        One-hot encoding is applied to columns specified in a pandas DataFrame.

        Modified from: https://gist.github.com/kljensen/5452382

        Details:

        http://en.wikipedia.org/wiki/One-hot
        http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

        @param df pandas DataFrame
        @param cols a list of columns to encode
        @return a DataFrame with one-hot encoding
        """
        vec = DictVectorizer()

        vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient='records')).toarray())
        vec_data.columns = vec.get_feature_names()
        vec_data.index = df.index

        df = df.drop(cols, axis=1)
        df = df.join(vec_data)
        self.vec_data_cols = vec_data.columns.values
                     
        return df
    
    
    def process_data(self, data):
#         data = data[data['SALE TYPE'] == 'PAST SALE']
        data = data.drop(RedfinModel.DROP_COLS, axis=1)
        self.numeric_cols = data.select_dtypes(include = ['float64', 'int64']).columns.values
        data = data.interpolate()
        data['LOT SIZE'] = data['LOT SIZE'].replace(np.NaN, 0)
        data['CITY'] = data['CITY'].astype(str).apply(lambda x: x.lower())
        data['PROPERTY TYPE'] = data['PROPERTY TYPE'].astype(str).apply(lambda x: x.lower())

        data = self.encode_onehot(data, RedfinModel.ENCODED_COLS)
        print('processed data columns: %s' % data.columns.values)
#         prop_mask = data['PROPERTY TYPE'].str.contains('5') | data['PROPERTY TYPE'].str.contains('vacant')
#         data = data[~prop_mask]
        if 'LOCATION' in data.columns.values:
            data = data.drop('LOCATION', axis=1)
            
        if 'STATE=WA' in data.columns.values:
            data = data.drop('STATE=WA', axis=1)
        
            
        location_cols = [x for x in data.columns.values if 'LOCATION' in x]
        for c in location_cols:
            if sum(data[c]) < 2:
                data = data.drop(c, axis=1)
                
        self.df = data
        return data

    def get_encodings(self, data):
        self.encoders = {}
        for c in RedfinModel.ENCODED_COLS:
            le = LabelEncoder()
            data[c] = data[c].apply(lambda x: x if not pd.isnull(x) else 'Null')
            le.fit(data[c])
            data[c] = le.transform(data[c])
            self.encoders[c] = le
    
    def transform(self, data):
        for c in RedfinModel.ENCODED_COLS:
            data[c] = data[c].apply(lambda x: x if not pd.isnull(x) else 'Null')
            data[c] = self.encoders[c].transform(data[c])
        return data
    
    def inverse_transform(self, data):
        for c in RedfinModel.ENCODED_COLS:
            data[c] = self.encoders[c].inverse_transform(data[c])
        return data
    
    def cat_correlation(self, category, transform_func=None):
        cat = self.df[category]
        if transform_func:
            cat = cat.apply(lambda x: transform_func(x))
        cat_df = pd.get_dummies(cat).join(self.df['PRICE']).astype(int)
        if 'nan' in cat_df.columns.values:
            del cat_df['nan']
        levels =  len(cat_df.columns.values)
        if levels > 60:
            print('%s: too many unique categories %d' % (category, levels))
            return cat_df
        corr = cat_df.corr()
        plt.figure(figsize=(16, 16))
        sns.heatmap(corr, vmax=1, square=True)
        return cat_df
    
    def model_cv_scores(self, d=4, est=250):
        clf = xgb.XGBRegressor(max_depth=d, n_estimators=est)
        clf.fit(self.X,self.y)
        scores = cross_val_score(clf, self.X, self.y, cv=5)
        print(scores, scores.mean())
        return clf
    
    def generate_model_data(self, d=4, est=200):
        # city_df = self.cat_correlation('CITY')
        # prop_df = self.cat_correlation('PROPERTY TYPE')
        full_df = self.df
        # full_df = self.df[redfin.numeric_cols].join(city_df.drop('PRICE', axis=1)).join(prop_df.drop('PRICE', axis=1))
        # full_df = full_df.join(sale_df.drop('PRICE', axis=1)).join(loc_df.drop('PRICE', axis=1))
        self.X = full_df.drop('PRICE', axis=1)
        self.y = full_df['PRICE']
        
    def predict_custom(self, data, d=4, est=200):
        self.model = Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression())])
#         self.model = LinearRegression()
        self.model.fit(self.X,self.y)
        # print(self.model.coef_)
        pred = self.model.predict(data)
        return pred
    
    def predict_lin(self, data, d=4, est=200):
        self.model = LinearRegression()
        self.model.fit(self.X,self.y)
        print(self.model.coef_)
        pred = self.model.predict(data)
        return pred
    
    def predict_xgb(self, data, d=4, est=200):
        self.clf = xgb.XGBRegressor(max_depth=d, n_estimators=est)
        self.clf.fit(self.X,self.y)
        pred = self.clf.predict(data)
        return pred

def convert_int(x):
    try:
        return int(x)
    except:
        return float('nan')
    
def convert_row_to_address(row):
    return "%s %s %s %s - %s, Delta Pred: %s" % (row['ADDRESS'], row['CITY'], row['STATE'],
                                   convert_int(row['ZIP']), row['PRICE'], row['Delta Pred'])
def print_rows(data, start, end):
    for i, row in data[start:end].iterrows():
        print(convert_row_to_address(row))

def process_train(data_file):
    train = pd.read_csv(data_file)
    train = train[(train['PRICE'] < 1.5*10**6) & (train['PRICE'] > 10**5) & (train['PROPERTY TYPE'] != 'Vacant Land')]
    return train
    
train = process_train(redfin_train_file)
print(train.columns.values)


np.set_printoptions(precision=2) #2 decimal places
np.set_printoptions(suppress=True) #remove scientific notation


In [ ]:
set(train['PROPERTY TYPE'])

In [ ]:
redfin = RedfinModel()
redfin.process_data(train)
redfin.df.info()
df = redfin.df

In [ ]:
corr = df.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(corr, vmax=1, square=True)

In [2]:
def add_numbers(x: int, y: int) -> int:
    '''Returns the sum of x and y'''
    return x + y

model = Model(add=add_numbers)

In [16]:
# Session Endpoints for Acumos Challenge model upload and auth.
session = AcumosSession(push_api="https://acumos-challenge.org/onboarding-app/v2/models",
                        auth_api="https://acumos-challenge.org/onboarding-app/v2/auth")

In [17]:
session.push(model, 'my-model')
session.dump(model, 'my-model', '~/')  # creates ~/my-model

AcumosError: Authentication failure: <!doctype html><html lang="en"><head><title>HTTP Status [500] – [Internal Server Error]</title><style type="text/css">h1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} h2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} h3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} body {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} b {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} p {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;} a {color:black;} a.name {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status [500] – [Internal Server Error]</h1><hr class="line" /><p><b>Type</b> Exception Report</p><p><b>Message</b> Request processing failed; nested exception is org.springframework.web.client.Htt

AcumosError: Authentication failure: <!doctype html><html lang="en"><head><title>HTTP Status [500] – [Internal Server Error]</title><style type="text/css">h1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} h2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} h3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} body {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} b {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} p {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;} a {color:black;} a.name {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status [500] – [Internal Server Error]</h1><hr class="line" /><p><b>Type</b> Exception Report</p><p><b>Message</b> Request processing failed; nested exception is org.springframework.web.client.Htt